In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.python.saved_model import loader
import os

tf.__version__
import tempfile
import subprocess
tf.contrib.lite.tempfile = tempfile
tf.contrib.lite.subprocess = subprocess

In [ ]:
saved_model_dir = 'saved_models/no_maxpool_10-5--10-5_120_1_0.01_0.95/1519994594'
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    with loaded_graph.as_default():
        loader.load(sess, ['serve'], saved_model_dir)
#         print(loaded_graph.get_operations())
        frozen_graph_def = tf.graph_util.convert_variables_to_constants(sess,
                                                                        loaded_graph.as_graph_def(),
                                                                        ["softmax_tensor"])
#     frozen_graph = tf.Graph()
#     with frozen_graph.as_default():
        tf.import_graph_def(frozen_graph_def)
#         print(loaded_graph.get_operations())
        graph_input = loaded_graph.get_operation_by_name('accelerometer_input')
        graph_output = loaded_graph.get_operation_by_name('softmax_tensor')
        input_tensor = graph_input.values()[0]
        input_shape = list(input_tensor.get_shape())
        print(input_shape)
        input_shape[0] = 1
        input_tensor.set_shape(input_shape)
        tflite_model = tf.contrib.lite.toco_convert(frozen_graph_def, graph_input.values(), [graph_output])

In [ ]:
saved_model_dir_head, _ = os.path.split(saved_model_dir)
_, name = os.path.split(saved_model_dir_head)
with open(os.path.join(saved_model_dir, '{}.tflite'.format(name)), 'wb') as f:
    f.write(tflite_model)

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer()
    tf.import_graph_def(frozen_graph_def)
    graph_input = sess.graph.get_operation_by_name('accelerometer_input')
    input_tensor = graph_input.values()[0]
    graph_output = sess.graph.get_operation_by_name('softmax_tensor')
    
    rand_data = np.random.rand(1, 120, 3)
    result = sess.run(graph_output, feed_dict={input_tensor: rand_data})
    print(result)